In [1]:
import os
import pandas as pd

In [2]:
# Get list of all the data files scrapped from the website
directory_path = "csv_files"
files = [f for f in os.listdir(directory_path) if os.path.isfile(os.path.join(directory_path, f))]

# pull out the table that actually has the information for the sites
sites_file = 'TABLE1_CompactionSites.csv'
files.remove(sites_file)

# Read sites into data frame
sites = pd.read_csv(os.path.join(directory_path, sites_file))

# add column to mathch the site name pulled from the site specific data file (i.e. no spaces)
sites['name_condensed'] = sites.apply(lambda x: x['GENERAL_NM'].replace(" ",""), axis=1)

In [3]:
sites_names = sites.name_condensed.unique()
sites_df = pd.DataFrame(sites_names, columns = ['name'])

In [5]:
sites.to_csv('compaction_sites.csv')

In [55]:
sites.head()

,SITE_NO,STATION_NM,COMPACTION_INTERVAL,ANCHOR_DEPTH,DEC_LONG_VA,DEC_LAT_VA,GENERAL_NM,name_condensed
0,292458094534206,KH-64-33-920 (Texas City Extensometer),CHICOT,800,-94.8950,29.41630,Texas City,TexasCity
1,295449095084105,LJ-65-07-909 (Lake Houston Extensometer),CHICOT AND EVANGELINE,"2,592",-95.1455,29.91320,Lake Houston,LakeHouston
2,294726095351102,LJ-65-12-726 (Addicks Extensometer),CHICOT AND EVANGELINE,"1,802",-95.5861,29.79070,Addicks,Addicks
3,294728095200106,LJ-65-14-746 (Northeast Extensometer),CHICOT AND EVANGELINE,"2,170",-95.3340,29.79090,Northeast,Northeast
4,294527095014910,LJ-65-16-930 (Baytown C-1 Extensometer),CHICOT,431,-95.0307,29.75784,Baytown Shallow,BaytownShallow


In [14]:
# Get master data file of all cumulative compaction for all sites, using site name in 'name_condensed' format as the column for future joining

In [82]:
for file in files:
    file_info = file.split('_')
    if file_info[1] not in sites_names:
        print('error: ', file)
        print(file_info)

error:  TABLE14_BaytownC2_Deep_2023.csv
['TABLE14', 'BaytownC2', 'Deep', '2023.csv']
error:  TABLE13_BaytownC1_Shallow_2023.csv
['TABLE13', 'BaytownC1', 'Shallow', '2023.csv']


In [93]:
name_replace_dict = {
    'BaytownC2_Deep': 'BaytownDeep',
    'BaytownC1_Shallow': 'BaytownShallow'
}
sites_data = pd.DataFrame(columns = ['DATE','CUMULATIVE_COMPACTION','site','data_version'])
for file in files:
    file_info = file
    for name in name_replace_dict.keys():
        if name in file:
            file_info = file_info.replace(name, name_replace_dict[name])         
    site_info = file_info.split('_')
    site_name = site_info[1]
    data_year = site_info[-1].replace('.csv','')
    df = pd.read_csv(os.path.join('csv_files',file))
    df['site'] = site_name
    df['data_version'] = data_year
    sites_data = pd.concat([sites_data, df], ignore_index=True)

/tmp/ipykernel_3118719/1850334796.py:17: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  sites_data = pd.concat([sites_data, df], ignore_index=True)


In [96]:
sites_data.to_csv('combined_site_data.csv')